In [ ]:
!pip install tensorflow
import os

# Defining each of these directories

In [ ]:
#base_dir = '../input/mechanical-tools-dataset/Mechanical Tools Image dataset'

train_dir = os.path.join('../input/mechanical-tools-dataset/train_data_V2/train_data_V2')
validation_dir = os.path.join('../input/mechanical-tools-dataset/validation_data/validation_data')

# Directory with our training screwdriver/wrench pictures
train_screwdriver_dir = os.path.join('../input/mechanical-tools-dataset/train_data_V2/train_data_V2/screwdriver')
train_wrench_dir = os.path.join('../input/mechanical-tools-dataset/train_data_V2/train_data_V2/wrench')

# Directory with our validation screwdriver/wrench pictures
validation_screwdriver_dir = os.path.join('../input/mechanical-tools-dataset/validation_data/validation_data/screwdriver')
validation_wrench_dir = os.path.join('../input/mechanical-tools-dataset/validation_data/validation_data/wrench')

**Now, let's see what the filenames look like in the training directories:**

In [ ]:
train_screwdriver_fnames = os.listdir(train_screwdriver_dir )
train_wrench_fnames = os.listdir( train_wrench_dir )

print(train_screwdriver_fnames[:20])
print(train_wrench_fnames[:20])

**Let's find out the number of wrench and pliers images in the directory**

In [ ]:
print('total training screwdriver images :', len(os.listdir(train_screwdriver_dir)))
print('total training wrench images :', len(os.listdir(train_wrench_dir)))

print('total validation screwdriver images :', len(os.listdir( validation_screwdriver_dir ) ))
print('total validation wrench images :', len(os.listdir( validation_wrench_dir) ))

**Now let's take a look at a few pictures to get a better sense of what they look like. First, configure the matplot parameters:**

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 10x10 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

**Now, display a batch of 50 wrench and 50 pliers pictures. You can rerun the cell to see a fresh batch each time:**

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_screwdriver_pix = [os.path.join(train_screwdriver_dir, fname) 
                for fname in train_screwdriver_fnames[pic_index-8:pic_index]]
next_wrench_pix = [os.path.join(train_wrench_dir, fname) 
                for fname in train_wrench_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_screwdriver_pix+next_wrench_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
import tensorflow as tf

**The "output shape" column shows how the size of your feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.**

# Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=30,
                                                    class_mode='binary',
                                                    target_size=(300, 300))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=30,
                                                         class_mode  = 'binary',
                                                         target_size = (300, 300))

**We then add convolutional layers as in the previous example, and flatten the final result to feed into the densely connected layers.**
**Note that because we are facing a two-class classification problem, i.e. a binary classification problem, we will end our network with a sigmoid activation, so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).**

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Conv2D(32, (4,4), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Conv2D(128,(4,4), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Conv2D(128,(4,4), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Conv2D(128,(4,4), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.30),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Dense(1, activation='sigmoid')  
]) 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('Screw driver') and 1 for the other ('Wrench')

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=15,
                              epochs=8,
                              validation_steps=15,
                              verbose=2)

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

In [ ]:
import cv2
import numpy as np
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1404).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screw driver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1407).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)

if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")


In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1408).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1410).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1415).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1401).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1402).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Wrench (286).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Wrench (293).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Wrench (276).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Wrench (268).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Wrench (313).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Wrench (324).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict_classes(img)

print(classes)
if classes[0]>0:
    print("This is a Wrench")
else :
    print("This is a Screwdriver")

In [ ]:
model.predict_classes(validation_generator)

**Visualizing Intermediate Representations
To get a feel for what kind of features our convnet has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the convnet.
Let's pick a random image from the training set, and then generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. Rerun this cell to generate intermediate representations for a variety of training images.**

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# Let's prepare a random input image from the training set.
wrench_img_files = [os.path.join(train_wrench_dir, f) for f in train_wrench_fnames]
screwdriver_img_files = [os.path.join(train_wrench_dir, f) for f in train_screwdriver_fnames]
img_path = random.choice(wrench_img_files + screwdriver_img_files)

img = load_img(img_path, target_size=(300, 300))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers[1:]]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
        n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
        size = feature_map.shape[1]
    # We will tile our images in this matrix
        display_grid = np.zeros((size, size * n_features))
        for i in range(n_features):
      # Postprocess the feature to make it visually palatable
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
            display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
        scale = 20. / n_features
        plt.figure(figsize=(scale * n_features, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='summer')